In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn.metrics
import scipy
import adjustText
import matplotlib.ticker

In [ ]:
# Set the default plot style
#default_plt_width = 15
#default_plt_height = 10
#plt.rcParams['figure.figsize'] = [default_plt_width, default_plt_height]

In [ ]:
sns.set_style("whitegrid")
sns.set_context("paper")
sns.set(font_scale=1.1)
sns.despine(left=True)
sns.set_style("ticks", {"xtick.major.size": 8, "ytick.major.size": 8})
cmap = sns.color_palette("Set1")
sns.palplot(cmap)
sns.set_palette(cmap)
plt_y_axis_fmt_string = '%.3f'

In [ ]:
filename_prefix = "aug_results_MNIST_3_vs_8_translate_10"

In [ ]:
# Parameters
filename_prefix = "aug_results_NORB_0_vs_1_crop_10"


In [ ]:
runs_data = np.load("{}.npz".format(filename_prefix))

In [ ]:
baseline_acc = runs_data["no_aug_no_poison_acc"]
poisoned_acc = runs_data["poisoned_acc"]
all_aug_train_poisoned_acc = runs_data["all_aug_train_poisoned_acc"]
n_aug_sample_points = runs_data["n_aug_sample_points"]
n_train = runs_data["n_train"]
VSV_acc = runs_data["VSV_acc"]
is_SV = runs_data["is_SV"].astype(np.int)
n_SV = np.sum(is_SV)

In [ ]:
runs_data_inf = pd.read_pickle("{}.pkl".format(filename_prefix))
runs_data_loss = pd.read_pickle("{}_loss.pkl".format(filename_prefix))

In [ ]:
runs_data_inf["score"] = "influence"

In [ ]:
runs_data_loss["score"] = "loss"

In [ ]:
run_df_unprocessed = pd.concat([
    runs_data_inf,
    runs_data_loss,
])

In [ ]:
run_df_unprocessed

In [ ]:
baseline = run_df_unprocessed.query("test_type == 'baseline'").reset_index()

In [ ]:
baseline["score"] = "baseline"

In [ ]:
baseline["test_type"] = "Baseline"

In [ ]:
baseline

In [ ]:
prop_inf = run_df_unprocessed.query("test_type == 'deterministic_proportional' & score == 'influence'")
prop_inf["test_type"] = "Deterministic Proportional Influence"

In [ ]:
prop_loss = run_df_unprocessed.query("test_type == 'deterministic_proportional' & score == 'loss'")
prop_loss["test_type"] = "Deterministic Proportional Loss"

In [ ]:
run_df = pd.concat([
    baseline,
    prop_inf,
    prop_loss,
])

In [ ]:
run_df = run_df.rename(
    index=str,
    columns={"test_accuracy": "Test Accuracy",
             "n_auged": "Number of Augmented Points",
            },
)

In [ ]:
run_df

In [ ]:
VSV_x = n_SV
VSV_y = VSV_acc

In [ ]:
fig, ax = plt.subplots()
run_plot = sns.lineplot(x="Number of Augmented Points",
                        y="Test Accuracy",
                        hue="test_type",
                        style="test_type",
                        ci=95,
                        data=run_df,
                        markers=True,
                        dashes=True,
                        ax=ax)
run_plot.scatter(VSV_x, VSV_y, marker="x", color="k", s=20)
#run_plot.annotate("VSV", (VSV_x - 73, VSV_y - 0.0008))
text = run_plot.text(VSV_x, VSV_y, "VSV", fontsize=12)
l = ax.legend()
#l.texts[0].set_text("")
#l.set_title('Whatever you want')
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles[1:], labels=labels[1:])
ax.yaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter(plt_y_axis_fmt_string))
plt.setp(ax.get_legend().get_texts(), fontsize='11.5') # for legend text 
#run_plot.axhline(y=baseline_acc,
#                 color="b",
#                 linestyle="--",
#                 label="baseline_acc")
run_plot.axhline(y=poisoned_acc,
                 color="r",
                 linestyle="--",
                 label="poisoned_acc")
run_plot.axhline(y=all_aug_train_poisoned_acc,
                 color="g",
                 linestyle="--",
                 label="all_aug_train_poisoned_acc")
adjustText.adjust_text([text],
                       x=[VSV_x],
                       y=[VSV_y],
                       add_objects=[run_plot],
                       expand_points=(0.2, 0.2),
                       expand_objects=(0.3, 0.3),
                       ax=ax,
                       force_objects=(0.1, 0.1))
run_plot.get_figure().savefig(filename_prefix + "_deterministic_joined.pdf",
                              bbox_inches="tight")